This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'sywTGuRk_YDDFwHhKCZh'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests 
from collections import namedtuple, OrderedDict

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key="+ API_KEY).json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

stockInfo = r['dataset']['data']
column_names = r['dataset']['column_names']

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:

# Store the data as a list of namedtuple instances 
# (one per day of info, with fields for date, 
# opening price, daily high, daily low, closing price, and daily trading volume)

stockInfoList=[]
stockInfoTuple=namedtuple('daily_info',['date','open','high','low','close','traded_volume'])

for dailyStock in stockInfo:
    stockInfoList.append(stockInfoTuple(dailyStock[0],dailyStock[1],dailyStock[2],dailyStock[3],dailyStock[4],dailyStock[6]))
    

In [6]:
# 3 - highest and lowest opening prices 

print("The highest opening price for the stock is $" + 
      str(max(stock.open for stock in stockInfoList if stock.open is not None)))

print("The lowest opening price for the stock is $" + 
      str(min(stock.open for stock in stockInfoList if stock.open is not None)))

The highest opening price for the stock is $53.11
The lowest opening price for the stock is $34.0


In [7]:
# 4 - largest change in any one day (based on High and Low price)

print("The largest change in any one day was $" + str(round(max(stock.high-stock.low for stock in stockInfoList),2)))

The largest change in any one day was $2.81


In [8]:
#5  largest change between any two days (based on Closing Price)

# The Quandl API has a parameter 'transform' that lets us calculate the day-by-day change in any of the 
# numeric columns, so we pull the change in the 5th column 'Close'. 

parameters={'api_key': API_KEY, 
            "start_date": '2017-01-01', 
            "end_date": '2017-12-31', 
            'order': 'asc', 
            'transform':'diff',
            'column_index':4}
r2=requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json",params=parameters)

print("The largest change between any two days was $" +
      str(max((abs(i[1])) for i in r2.json()['dataset_data']['data'])))

The largest change between any two days was $2.56


In [9]:
#6 What was the average daily trading volume during this year?

print("The average daily trading volume during this year was $" +
      str(round(sum(stock.traded_volume for stock in stockInfoList)/len(stockInfoList),2)))

The average daily trading volume during this year was $89124.34
